In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/ubuntu/varios/skforecast


In [12]:
# Unit test bayesian_search_forecaster_multiseries
# ==============================================================================
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series


from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import custom_metric
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import expected_df_to_long_format
THIS_DIR = Path('/home/ubuntu/varios/skforecast/skforecast/model_selection/tests')
series_item_sales = pd.read_parquet(THIS_DIR/'fixture_multi_series_items_sales.parquet')
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [20]:
forecaster = ForecasterRecursiveMultiSeries(
    regressor=LGBMRegressor(
        n_estimators=30, random_state=123, verbose=-1, max_depth=4
    ),
    lags=[1, 7, 14],
    encoding=None,
    differentiation=1,
    dropna_from_series=False,
    transformer_series=None,
    transformer_exog=StandardScaler(),
)

cv = TimeSeriesFold(
            initial_train_size = len(series_dict_train['id_1000']),
            steps              = 24,
            refit              = False,
            fixed_train_size   = True,
            gap                = 0,
            differentiation    = forecaster.differentiation_max
        )

metrics, predictions = backtesting_forecaster_multiseries(
    forecaster        = forecaster,
    series            = series_dict,
    exog              = exog_dict,
    cv                = cv,
    metric            = ['mean_absolute_error', 'mean_absolute_scaled_error'],
    n_jobs            = 'auto',
    verbose           = True,
    show_progress     = True,
    suppress_warnings = True
)

expected_metrics = pd.DataFrame(
    data={
        'levels': [
            'id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004',
            'average', 'weighted_average', 'pooling'
        ],
        'mean_absolute_error': [
            234.51032919, 1145.83513569, np.nan, 1342.06986733,
            1025.76779699, 937.0457823, 818.85514869, 818.85514869
        ],
        'mean_absolute_scaled_error': [
            1.08353766, 3.16716567, np.nan, 5.4865466 ,
            0.8435216 , 2.64519288, 2.55539804, 1.98180886
        ]
    }
)
expected_predictions = pd.DataFrame(
    data=np.array([
        [1351.44874045, 3267.13419659, 3843.06135374, 7220.15909652],
        [1411.73786344, 3537.21728977, 3823.777024  , 7541.54119992],
        [1367.56233886, 3595.37617537, 3861.96220571, 7599.93110962],
        [1349.98221742, 3730.36025071, 3900.14738742, 7997.86353537],
        [1330.33321825, 3752.7395927 , 3850.39111741, 7923.44398373],
        [1045.2013386 , 2652.15007539, 4227.38385608, 6651.39787084],
        [ 801.6084919 , 2059.57604816,        np.nan, 5625.61804572],
        [1473.95692547, 3159.99921219,        np.nan, 6740.53028097],
        [1596.51499989, 3449.40467366,        np.nan, 7172.21235605],
        [1572.31317919, 3507.56355926,        np.nan, 7506.35124681]]),
    index=pd.date_range('2016-08-01', periods=10, freq='D'),
    columns=['id_1000', 'id_1001', 'id_1003', 'id_1004']
)
expected_predictions = expected_df_to_long_format(expected_predictions)

pd.testing.assert_frame_equal(metrics, expected_metrics)
pd.testing.assert_frame_equal(predictions.head(40), expected_predictions)

Information of folds
--------------------
Number of observations used for initial training: 212
    First 1 observation/s in training sets are used for differentiation
Number of observations used for backtesting: 153
    Number of folds: 7
    Number skipped folds: 0 
    Number of steps per fold: 24
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 9 observations.

Fold: 0
    Training:   2016-01-02 00:00:00 -- 2016-07-31 00:00:00  (n=212)
    Validation: 2016-08-01 00:00:00 -- 2016-08-24 00:00:00  (n=24)
Fold: 1
    Training:   No training in this fold
    Validation: 2016-08-25 00:00:00 -- 2016-09-17 00:00:00  (n=24)
Fold: 2
    Training:   No training in this fold
    Validation: 2016-09-18 00:00:00 -- 2016-10-11 00:00:00  (n=24)
Fold: 3
    Training:   No training in this fold
    Validation: 2016-10-12 00:00:00 -- 2016-11-04 00:00:00  (n=24)
Fold: 4
    Training:   No training in this fold
    Validation: 

  0%|          | 0/7 [00:00<?, ?it/s]

AssertionError: DataFrame.iloc[:, 2] (column name="mean_absolute_scaled_error") are different

DataFrame.iloc[:, 2] (column name="mean_absolute_scaled_error") values are different (50.0 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7]
[left]:  [1.0835376623490711, 3.1671656660510625, nan, 5.4872216266448595, 0.8435215951118011, 2.645361637539198, 2.555504352930781, 1.9838266411338281]
[right]: [1.08353766, 3.16716567, nan, 5.4865466, 0.8435216, 2.64519288, 2.55539804, 1.98180886]
At positional index 3, first diff: 5.4872216266448595 != 5.4865466

In [21]:
metrics

,levels,mean_absolute_error,mean_absolute_scaled_error
0,id_1000,234.510329,1.083538
1,id_1001,1145.835136,3.167166
2,id_1002,NaN,NaN
3,id_1003,1342.069867,5.487222
4,id_1004,1025.767797,0.843522
5,average,937.045782,2.645362
6,weighted_average,818.855149,2.555504
7,pooling,818.855149,1.983827


In [22]:
expected_metrics

,levels,mean_absolute_error,mean_absolute_scaled_error
0,id_1000,234.510329,1.083538
1,id_1001,1145.835136,3.167166
2,id_1002,NaN,NaN
3,id_1003,1342.069867,5.486547
4,id_1004,1025.767797,0.843522
5,average,937.045782,2.645193
6,weighted_average,818.855149,2.555398
7,pooling,818.855149,1.981809
